### Install the package

In [ ]:
# !git clone https://github.com/mohalisad/Cough-signal-processing.git
!pip install -r ./Cough-signal-processing/requirements.txt
!mv ./Cough-signal-processing/csp ./

### Start the code

In [1]:
def save_as_image(np_array, file_name, size=(224, 224)):
    np_array = np_array - np_array.min()
    np_array = np_array / np_array.max()
    np_array = np_array[::-1, :]

    im = Image.fromarray(np.uint8(cm.jet(np_array) * 255)).resize(size)
    im.save(np_array)

NameError: name 'SpectrogramFeatures' is not defined

In [12]:
import json

for item in all_data.values():
    for k, v in item.items():
        if type(v) is np.ndarray:
            item[k] = v.tolist()

with open('features.json', 'w') as f:
    json.dump(all_data, f)

In [ ]:
import glob
from pathlib import Path
from csp import SpectrogramFeatures
from tqdm.notebook import tqdm
import numpy as np
from csp import time_domain
from PIL import Image
from matplotlib import cm

folders = ['./data/longitudinal/', './data/solicited/']

OUTPUT_PREFIX = './output'

all_data = {}

for folder in folders:
    for filename in tqdm(glob.glob(f'{folder}*.wav')):
        p = Path(filename)
        file_id = p.name[:-4]
        folder = p.parent.name
        data = SpectrogramFeatures(filename).spectrogram_data()
        
        mag_path = Path(f'{OUTPUT_PREFIX}/{folder}_mag/{file_id}.npy')
        mag_path_png = Path(f'{OUTPUT_PREFIX}/{folder}_mag_png/{file_id}.png')
        mel_mag_path = Path(f'{OUTPUT_PREFIX}/{folder}_mel_mag/{file_id}.npy')
        mel_mag_path_png = Path(f'{OUTPUT_PREFIX}/{folder}_mel_mag_png/{file_id}.png')
        
        for p in [mag_path, mag_path_png, mel_mag_path, mel_mag_path_png]:
            p.parent.mkdir(parents=True, exist_ok=True)
        
        with open(mag_path, 'wb') as f:
            np.save(f, data['log_spectrogram'])
        save_as_image(data['log_spectrogram'], mag_path_png)
        
        with open(mel_mag_path, 'wb') as f:
            np.save(f, data['mel_spectrogram_db'])
        save_as_image(data['mel_spectrogram_db'], mel_mag_path_png)
            
        features = {
            'ZCR': time_domain.zero_crossing_rate(data['signal']),
            'LogE': time_domain.logenergy_computation(data['signal']),
        }
        mfcc_features = time_domain.melfrequency_cepstral_coefficients('lib', *([None] * 5), sound_file=p)
        all_data[str(p)] = {
            **mfcc_features,
            **features
        }